### initialization

In [1]:
from pathlib import Path
SRC_PATH = Path('/home/ec2-user/SageMaker/david/tdm-sentiment/src/')
import sys
sys.path.append(str(SRC_PATH))
from config import *
import pandas as pd
import logging

is_economic_model = is_economic_module.EconomicClassifier(model_path=IS_ECONOMIC_MODEL, initialize=True)  # Initialize the economic model
tdm_parser = tdm_parser_module.TdmXmlParser()

corpuses_dir = CORPUSES_PATH  #'all_dataset_file_names.txt' LosAngelesTimesDavid all_dataset_file_names.txt # Path to the input
output_path = FILE_NAMES_PATH / 'is_economic_train_files.txt'  # Path to the output text file
csv_path = PROJECT_DATA_PATH / 'train_data' / 'economic_classifier_train_data.csv'
file_names_path = FILE_NAMES_PATH / 'is_economic_train_files.txt'  # 'all_dataset_file_names.txt'  # Path to the file containing names


# set logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers.clear()

handler = logging.FileHandler(filename=LOGS_PATH / 'train_is_economic_info.log') 
formatter =logging.Formatter('%(asctime)s - %(name)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

/home/davidrozenblat/tdm-sentiment/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### train model 

In [2]:
# try get data training dataset if exists
try:
    df = pd.read_csv(csv_path)
    print(df.head(2))
except:    
    # get sample file
    file_process.get_file_paths_sample(corpuses_dir, output_path, sample_size=500*10**3)

    # crate and save df
    df = file_process.load_df_from_xml(corpuses_dir, file_names_path)
    # save df
    csv_path.parent.mkdir(parents=True, exist_ok=True) # Ensure the output directory exists
    df.to_csv(csv_path, index=False)
    logger.info(f"Training data saved to {csv_path}")
    #print(df.head(2))

# Assuming df is a DataFrame with the necessary columns
X_test, y_test = is_economic_model.train_classifier(df, data_column='paragrph_text', label_column='section')
is_economic_model.save_model() # be careful to make a copy of current one first 

logger.info(f'model evaluation with thrashold=0.5 is: \n {is_economic_model.evaluate_model(X_test, y_test, threshold=0.5)}')
logger.info(f'model evaluation with thrashold=0.7 is: \n {is_economic_model.evaluate_model(X_test, y_test, threshold=0.7)}')

         goid          publisher        date         article_type  \
0   422244975  Los Angeles Times  2008-11-02  General Information   
1  2869108043  Los Angeles Times  2023-09-27                 News   

                                  section tdm_topic_tags page  word_count  \
0  Sunday Calendar; Part E; Calendar Desk            NaN  E.2          39   
1         California; Part B; Sports Desk       Scandals  B.9         187   

                  title                                      paragrph_text  \
0        For the record  "mad men" actress: a caption under photographs...   
1  SPORTS; ON THIS DATE  1950 - ezzard charles wins a unanimous 15-roun...   

  company_name  
0          NaN  
1          NaN  
Preprocess error: 'float' object has no attribute 'lower'


/home/davidrozenblat/tdm-sentiment/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidrozenblat/tdm-sentiment/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/davidrozenblat/tdm-sentiment/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m